#                                             IBM Data Science Capstone Project Final Notebook

#### I will create clusters to find the most suitable location to open a luxury Asian restaurant in Paris, France

### import of libraries 

In [1]:
! pip install folium==0.5.0
! pip install geopy

     |████████████████████████████████| 79 kB 9.7 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=8a3c83e4728b26d411944709539748d41eabf76385aa147f85b318de5cb4e28c
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [2]:
import numpy as np # library to handle data in a vectorized manner
import json # library to handle JSON files
import requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


## DATA Processing

### Setting up of the Paris dataset 

#### JSON data of France

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_241eaa55d6944ad0a1727cec221c3413 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='j3Vt4x2pvma42CubkNX7tlxhiCki8sijMXH2t-xDPrmA',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_241eaa55d6944ad0a1727cec221c3413.get_object(Bucket='capstoneprojectcoursera-donotdelete-pr-fscs4t6kybtjao',Key='correspondances-code-insee-code-postal.json')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face an error during data loading.
# Please read the documentation of 'pandas.read_json()' and 'pandas.io.json.json_normalize' to learn more about the possibilities to adjust the data loading.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html

df = pd.read_json(body, orient='values')
df.head()


,datasetid,recordid,fields,geometry,record_timestamp
0,correspondances-code-insee-code-postal,2bf36b38314b6c39dfbcd09225f97fa532b1fc45,"{'code_comm': '645', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.2517129721...",2016-09-21T00:29:06.175+02:00
1,correspondances-code-insee-code-postal,7ee82e74e059b443df18bb79fc5a19b1f05e5a88,"{'code_comm': '133', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [3.0529405055...",2016-09-21T00:29:06.175+02:00
2,correspondances-code-insee-code-postal,e2cd3186f07286705ed482a10b6aebd9de633c81,"{'code_comm': '378', 'nom_dept': 'ESSONNE', 's...","{'type': 'Point', 'coordinates': [2.1971816504...",2016-09-21T00:29:06.175+02:00
3,correspondances-code-insee-code-postal,868bf03527a1d0a9defe5cf4e6fa0a730d725699,"{'code_comm': '243', 'nom_dept': 'SEINE-ET-MAR...","{'type': 'Point', 'coordinates': [2.7097808131...",2016-09-21T00:29:06.175+02:00
4,correspondances-code-insee-code-postal,21e809b1d4480333c8b6fe7addd8f3b06f343e2c,"{'code_comm': '003', 'nom_dept': 'VAL-DE-MARNE...","{'type': 'Point', 'coordinates': [2.3335102498...",2016-09-21T00:29:06.175+02:00


In [54]:
#subset of France dataset which contains fields of france data
df2 = pd.DataFrame()
for element in df.fields:
    df2 = df2.append(element, ignore_index=True)
print(df2.shape)
df2.head()

(1300, 17)


,code_arr,code_cant,code_comm,code_dept,code_reg,geo_point_2d,geo_shape,id_geofla,insee_com,nom_comm,nom_dept,nom_region,population,postal_code,statut,superficie,z_moyen
0,3,03,645,91,11,"[48.750443119964764, 2.251712972144151]","{'type': 'Polygon', 'coordinates': [[[2.238024...",16275,91645,VERRIERES-LE-BUISSON,ESSONNE,ILE-DE-FRANCE,15.5,91370,Commune simple,999.0,121.0
1,3,20,133,77,11,"[48.41256065214989, 3.052940505560729]","{'type': 'Polygon', 'coordinates': [[[3.076046...",31428,77133,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,ILE-DE-FRANCE,0.2,77126,Commune simple,1082.0,88.0
2,1,09,378,91,11,"[48.52726809075556, 2.19718165044305]","{'type': 'Polygon', 'coordinates': [[[2.203466...",30975,91378,MAUCHAMPS,ESSONNE,ILE-DE-FRANCE,0.3,91730,Commune simple,313.0,150.0
3,5,14,243,77,11,"[48.87307018579678, 2.7097808131278462]","{'type': 'Polygon', 'coordinates': [[[2.727542...",17000,77243,LAGNY-SUR-MARNE,SEINE-ET-MARNE,ILE-DE-FRANCE,20.2,77400,Chef-lieu canton,579.0,71.0
4,3,34,003,94,11,"[48.80588035965699, 2.333510249842654]","{'type': 'Polygon', 'coordinates': [[[2.343851...",32123,94003,ARCUEIL,VAL-DE-MARNE,ILE-DE-FRANCE,19.5,94110,Chef-lieu canton,232.0,70.0


In [55]:
#subset of France dataset which contains neighbourhoods, postal code, latitude and longitude data
df3 = df2[['postal_code','nom_comm','nom_dept','geo_point_2d']]
df3.head()

,postal_code,nom_comm,nom_dept,geo_point_2d
0,91370,VERRIERES-LE-BUISSON,ESSONNE,"[48.750443119964764, 2.251712972144151]"
1,77126,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,"[48.41256065214989, 3.052940505560729]"
2,91730,MAUCHAMPS,ESSONNE,"[48.52726809075556, 2.19718165044305]"
3,77400,LAGNY-SUR-MARNE,SEINE-ET-MARNE,"[48.87307018579678, 2.7097808131278462]"
4,94110,ARCUEIL,VAL-DE-MARNE,"[48.80588035965699, 2.333510249842654]"


In [56]:
# dataset of Paris
df_paris = df3[df3['nom_dept'].str.contains('PARIS')].reset_index(drop=True)
df_paris.head()

,postal_code,nom_comm,nom_dept,geo_point_2d
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]"
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]"
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]"
3,75015,PARIS-15E-ARRONDISSEMENT,PARIS,"[48.84015541860987, 2.293559372435076]"
4,75003,PARIS-3E-ARRONDISSEMENT,PARIS,"[48.86305413181178, 2.359361058970589]"


In [57]:
df_paris['geo_point_2d'][0] 

[48.87689616237872, 2.337460241388529]

### We have to split the geo_point_2d column into latitude and longitude

In [58]:
lat_lng = df_paris['geo_point_2d'].astype('str') # we have to be sure that is a str variable in order to split it

# latitude
lat = lat_lng.apply(lambda x: x.split(',')[0])
lat = lat.apply(lambda x: x.lstrip('['))

# longitude
lng = lat_lng.apply(lambda x: x.split(',')[1])
lng = lng.apply(lambda x: x.rstrip(']'))

#### I create latitude and longitude data frames that I add to the Paris data frame

In [59]:
paris_lat  = pd.DataFrame(paris_lat.astype(float))
paris_lat.columns=['Latitude']

paris_lng = pd.DataFrame(paris_lng.astype(float))
paris_lng.columns=['Longitude']

paris_f = pd.concat([df_paris.drop('geo_point_2d', axis=1), paris_lat, paris_lng], axis=1)
print(paris_f.shape)
paris_f.head() 

(20, 5)


,postal_code,nom_comm,nom_dept,Latitude,Longitude
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741
3,75015,PARIS-15E-ARRONDISSEMENT,PARIS,48.840155,2.293559
4,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361


## Let's visualize the Map of Paris

In [8]:
address = 'Paris'
geolocator = Nominatim(user_agent="paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


In [60]:
# Creating the map of Paris
map_Paris= folium.Map(location=[latitude, longitude], zoom_start=12)
map_Paris

# adding markers to map
for latitude, longitude, borough, town in zip(paris_f['Latitude'], paris_f['Longitude'], paris_f['nom_comm'], paris_f['nom_dept']):
    label = '{}, {}'.format(town, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='Blue',
        fill=True,
        fill_opacity=0.8
        ).add_to(map_Paris)  
    
map_Paris

## Getting Venues Data using Foursquare

In [10]:
CLIENT_ID = 'MRO4ZY41CTLF24NWWE0IWDJL3AQDOYVBVCTCS4KQSIM4XIFV' 
CLIENT_SECRET = '0ZHM1TMNMUE5WC2ZMVDCTQRQSG5KYHRFVPLCQITLSMRROXSP' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MRO4ZY41CTLF24NWWE0IWDJL3AQDOYVBVCTCS4KQSIM4XIFV
CLIENT_SECRET:0ZHM1TMNMUE5WC2ZMVDCTQRQSG5KYHRFVPLCQITLSMRROXSP


In [61]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [62]:
#Get venues for all neighborhoods in our dataset
paris_venues = getNearbyVenues(names=paris_f['nom_comm'], latitudes=paris_f['Latitude'], longitudes=paris_f['Longitude'])

PARIS-9E-ARRONDISSEMENT
PARIS-2E-ARRONDISSEMENT
PARIS-11E-ARRONDISSEMENT
PARIS-15E-ARRONDISSEMENT
PARIS-3E-ARRONDISSEMENT
PARIS-6E-ARRONDISSEMENT
PARIS-5E-ARRONDISSEMENT
PARIS-19E-ARRONDISSEMENT
PARIS-20E-ARRONDISSEMENT
PARIS-1ER-ARRONDISSEMENT
PARIS-17E-ARRONDISSEMENT
PARIS-8E-ARRONDISSEMENT
PARIS-13E-ARRONDISSEMENT
PARIS-12E-ARRONDISSEMENT
PARIS-18E-ARRONDISSEMENT
PARIS-10E-ARRONDISSEMENT
PARIS-16E-ARRONDISSEMENT
PARIS-4E-ARRONDISSEMENT
PARIS-7E-ARRONDISSEMENT
PARIS-14E-ARRONDISSEMENT


## Analyse Neighbourhoods of Paris

In [63]:
print(paris_venues.shape)
paris_venues.head()

(1306, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,So Nat,48.876277,2.338614,Vegetarian / Vegan Restaurant
1,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Place Saint-Georges,48.878408,2.337575,Plaza
2,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Farine & O,48.877209,2.339464,Bakery
3,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Le Bouclier de Bacchus,48.876834,2.337843,Wine Bar
4,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,La Compagnie du Café,48.877916,2.337997,Café


In [64]:
#Number of venues per neighborhood
paris_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
PARIS-10E-ARRONDISSEMENT,100,100,100,100,100,100
PARIS-11E-ARRONDISSEMENT,42,42,42,42,42,42
PARIS-12E-ARRONDISSEMENT,5,5,5,5,5,5
PARIS-13E-ARRONDISSEMENT,58,58,58,58,58,58
PARIS-14E-ARRONDISSEMENT,26,26,26,26,26,26
PARIS-15E-ARRONDISSEMENT,57,57,57,57,57,57
PARIS-16E-ARRONDISSEMENT,11,11,11,11,11,11
PARIS-17E-ARRONDISSEMENT,55,55,55,55,55,55
PARIS-18E-ARRONDISSEMENT,71,71,71,71,71,71


In [65]:
#Number of unique venue categories
print('There are {} uniques categories.'.format(len(paris_venues['Venue Category'].unique())))

There are 204 uniques categories.


In [66]:
#print out the list of categories
paris_venues['Venue Category'].unique()[:100]

array(['Vegetarian / Vegan Restaurant', 'Plaza', 'Bakery', 'Wine Bar',
       'Café', 'Venezuelan Restaurant', 'Gourmet Shop',
       'Gym / Fitness Center', 'French Restaurant', 'Bistro',
       'Seafood Restaurant', 'Lounge', 'Latin American Restaurant',
       'Museum', 'Falafel Restaurant', 'Brazilian Restaurant',
       'Japanese Restaurant', 'Tea Room', 'Chinese Restaurant',
       'Vietnamese Restaurant', 'Pizza Place', 'Restaurant',
       'Cheese Shop', 'Hotel', 'Coffee Shop', 'Deli / Bodega',
       'Gift Shop', 'Cocktail Bar', 'Modern European Restaurant',
       'Chocolate Shop', 'Bar', 'Corsican Restaurant', 'Snack Place',
       'Burger Joint', 'Shoe Store', 'Bed & Breakfast', 'Breakfast Spot',
       'Creperie', 'Theater', 'Juice Bar', 'Turkish Restaurant',
       'African Restaurant', 'Candy Store', 'Ice Cream Shop',
       'Antique Shop', 'Dessert Shop', 'Sandwich Place',
       'Italian Restaurant', 'Mexican Restaurant', 'Salad Place',
       'Cupcake Shop', 'Escape R

In [67]:
# check if the results contain "Asian Restaurant"
"Asian Restaurant" in paris_venues['Venue Category'].unique()

True

### One hot encoding

In [68]:
onehot = pd.get_dummies(paris_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhoods'] = paris_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
onehot = to_onehot[fixed_columns]

print(onehot.shape)
onehot.head()

(1306, 205)


,Zoo Exhibit,Neighborhoods,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo
0,0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,PARIS-9E-ARRONDISSEMENT,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
paris_grouped = onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(paris_grouped.shape)
paris_grouped

(20, 205)


,Neighborhoods,Zoo Exhibit,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo
0,PARIS-10E-ARRONDISSEMENT,0.0,0.00000,0.02,0.000000,0.00,0.00,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.020000,0.020000,0.000000,0.0
1,PARIS-11E-ARRONDISSEMENT,0.0,0.02381,0.00,0.000000,0.00,0.00,0.00000,0.023810,0.000000,...,0.000000,0.000000,0.000000,0.047619,0.00,0.023810,0.023810,0.000000,0.000000,0.0
2,PARIS-12E-ARRONDISSEMENT,0.2,0.00000,0.00,0.000000,0.00,0.00,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.2
3,PARIS-13E-ARRONDISSEMENT,0.0,0.00000,0.00,0.000000,0.00,0.00,0.00000,0.000000,0.000000,...,0.017241,0.000000,0.000000,0.000000,0.00,0.206897,0.000000,0.000000,0.000000,0.0
4,PARIS-14E-ARRONDISSEMENT,0.0,0.00000,0.00,0.000000,0.00,0.00,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
5,PARIS-15E-ARRONDISSEMENT,0.0,0.00000,0.00,0.000000,0.00,0.00,0.00000,0.000000,0.017544,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
6,PARIS-16E-ARRONDISSEMENT,0.0,0.00000,0.00,0.000000,0.00,0.00,0.00000,0.090909,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
7,PARIS-17E-ARRONDISSEMENT,0.0,0.00000,0.00,0.000000,0.00,0.00,0.00000,0.018182,0.000000,...,0.000000,0.018182,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0
8,PARIS-18E-ARRONDISSEMENT,0.0,0.00000,0.00,0.014085,0.00,0.00,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.014085,0.00,0.028169,0.014085,0.000000,0.000000,0.0
9,PARIS-19E-ARRONDISSEMENT,0.0,0.00000,0.00,0.023256,0.00,0.00,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00,0.023256,0.000000,0.000000,0.000000,0.0


#### We have to check the number of Asian Restaurant 

In [35]:
len(paris_grouped[paris_grouped["Asian Restaurant"] > 0])

6

In [70]:
asian = paris_grouped[["Neighborhoods","Asian Restaurant"]]
asian.head()

,Neighborhoods,Asian Restaurant
0,PARIS-10E-ARRONDISSEMENT,0.030000
1,PARIS-11E-ARRONDISSEMENT,0.023810
2,PARIS-12E-ARRONDISSEMENT,0.000000
3,PARIS-13E-ARRONDISSEMENT,0.189655
4,PARIS-14E-ARRONDISSEMENT,0.000000


### Let's check the top venues in Paris

In [71]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [72]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [74]:
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = paris_grouped['Neighborhoods']

for ind in np.arange(paris_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(paris_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-10E-ARRONDISSEMENT,French Restaurant,Coffee Shop,Hotel,Bistro,Café,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Asian Restaurant,Pizza Place
1,PARIS-11E-ARRONDISSEMENT,Restaurant,Café,French Restaurant,Cocktail Bar,Italian Restaurant,Bistro,Bakery,Vegetarian / Vegan Restaurant,Pastry Shop,Mediterranean Restaurant
2,PARIS-12E-ARRONDISSEMENT,Zoo,Supermarket,Bistro,Monument / Landmark,Zoo Exhibit,Venezuelan Restaurant,Food & Drink Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
3,PARIS-13E-ARRONDISSEMENT,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,French Restaurant,Juice Bar,Hotel,Plaza,Coffee Shop,Park
4,PARIS-14E-ARRONDISSEMENT,French Restaurant,Hotel,Food & Drink Shop,Bistro,Japanese Restaurant,Café,Tea Room,Fast Food Restaurant,Bakery,Brasserie
5,PARIS-15E-ARRONDISSEMENT,Italian Restaurant,Hotel,French Restaurant,Brasserie,Coffee Shop,Restaurant,Lebanese Restaurant,Bistro,Japanese Restaurant,Thai Restaurant
6,PARIS-16E-ARRONDISSEMENT,Plaza,Lake,French Restaurant,Art Museum,Park,Bus Stop,Bus Station,Boat or Ferry,Pool,Historic Site
7,PARIS-17E-ARRONDISSEMENT,Hotel,French Restaurant,Italian Restaurant,Bakery,Japanese Restaurant,Café,Plaza,Bistro,Restaurant,Concert Hall
8,PARIS-18E-ARRONDISSEMENT,French Restaurant,Bar,Pizza Place,Italian Restaurant,Bistro,Restaurant,Plaza,Café,Hotel,Convenience Store
9,PARIS-19E-ARRONDISSEMENT,French Restaurant,Bar,Hotel,Supermarket,Seafood Restaurant,Brewery,Bistro,Beer Bar,Creperie,Sandwich Place


## Cluster Neighborhoods of Paris

#### I run k-means to cluster the neighborhood of Paris into 3 clusters.

In [76]:
# set number of clusters
kclusters = 3

to_clustering = asian.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 2, 0, 1, 0, 0, 0, 0, 0, 0], dtype=int32)

In [77]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
asian_merged = african.copy()

# add clustering labels
asian_merged["Cluster Labels"] = kmeans.labels_

In [78]:
asian_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True) 
asian_merged.head()

,Neighborhood,Asian Restaurant,Cluster Labels
0,PARIS-10E-ARRONDISSEMENT,0.030000,2
1,PARIS-11E-ARRONDISSEMENT,0.023810,2
2,PARIS-12E-ARRONDISSEMENT,0.000000,0
3,PARIS-13E-ARRONDISSEMENT,0.189655,1
4,PARIS-14E-ARRONDISSEMENT,0.000000,0


In [79]:
# merge asian merge data with paris venues data, due to the Neighborhood column
asian_merged = asian_merged.join(paris_venues.set_index("Neighborhood"), on="Neighborhood")
print(to_merged.shape)
to_merged.head()

(1306, 9)


,Neighborhood,Asian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,PARIS-2E-ARRONDISSEMENT,0.01,0,48.867903,2.344107,Cloud Cakes,48.865641,2.346302,Bakery
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Quai d'Orléans,48.851596,2.354282,Trail
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Galerie Azzedine Alaïa,48.857545,2.355217,Art Gallery
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Les Nautes,48.852231,2.360306,Gastropub
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Pitzman,48.855853,2.360295,Falafel Restaurant


In [80]:
# sort the results by Cluster Labels
asian_merged.sort_values(["Cluster Labels"], inplace=True)
asian_merged

,Neighborhood,Asian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,PARIS-2E-ARRONDISSEMENT,0.01,0,48.867903,2.344107,Cloud Cakes,48.865641,2.346302,Bakery
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Quai d'Orléans,48.851596,2.354282,Trail
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Galerie Azzedine Alaïa,48.857545,2.355217,Art Gallery
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Les Nautes,48.852231,2.360306,Gastropub
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Pitzman,48.855853,2.360295,Falafel Restaurant
...,...,...,...,...,...,...,...,...,...
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,Bombay Palace,48.878327,2.356484,Indian Restaurant
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,La Grange,48.875331,2.367360,Italian Restaurant
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,Midnight Hotel,48.877301,2.356188,Hotel
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,Sushi Rama,48.873197,2.358073,Japanese Restaurant


## let's visualize the cluster

In [81]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(asian_merged['Neighborhood Latitude'], asian_merged['Neighborhood Longitude'], asian_merged['Neighborhood'], asian_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the 3 clusters

In [82]:
#Cluster 0
asian_merged.loc[asian_merged['Cluster Labels'] == 0]

,Neighborhood,Asian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,PARIS-2E-ARRONDISSEMENT,0.01,0,48.867903,2.344107,Cloud Cakes,48.865641,2.346302,Bakery
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Quai d'Orléans,48.851596,2.354282,Trail
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Galerie Azzedine Alaïa,48.857545,2.355217,Art Gallery
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Les Nautes,48.852231,2.360306,Gastropub
14,PARIS-4E-ARRONDISSEMENT,0.00,0,48.854228,2.357362,Pitzman,48.855853,2.360295,Falafel Restaurant
...,...,...,...,...,...,...,...,...,...
10,PARIS-1ER-ARRONDISSEMENT,0.00,0,48.862630,2.336293,"Pavillon des Sessions – Arts d'Afrique, d'Asie...",48.860724,2.332121,Art Museum
10,PARIS-1ER-ARRONDISSEMENT,0.00,0,48.862630,2.336293,Le Café Blanc,48.862719,2.339578,Bar
10,PARIS-1ER-ARRONDISSEMENT,0.00,0,48.862630,2.336293,Jardins du Carrousel (Jardin du Carrousel),48.862056,2.332445,Garden
10,PARIS-1ER-ARRONDISSEMENT,0.00,0,48.862630,2.336293,Christian Louboutin,48.862697,2.340757,Shoe Store


In [83]:
#Cluster 1
asian_merged.loc[asian_merged['Cluster Labels'] == 1]

,Neighborhood,Asian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
3,PARIS-13E-ARRONDISSEMENT,0.189655,1,48.828718,2.362468,Comme Au Vietnam,48.829701,2.357442,Vietnamese Restaurant
3,PARIS-13E-ARRONDISSEMENT,0.189655,1,48.828718,2.362468,Le Grenier à Pain,48.827672,2.356468,Bakery
3,PARIS-13E-ARRONDISSEMENT,0.189655,1,48.828718,2.362468,Au Village de Choisy,48.825478,2.360682,Chinese Restaurant
3,PARIS-13E-ARRONDISSEMENT,0.189655,1,48.828718,2.362468,Bubble Tea,48.826289,2.359718,Juice Bar
3,PARIS-13E-ARRONDISSEMENT,0.189655,1,48.828718,2.362468,Crêperie Paris Breizh,48.831965,2.358195,Creperie
3,PARIS-13E-ARRONDISSEMENT,0.189655,1,48.828718,2.362468,Olympiades Entrée Principale,48.826598,2.364797,Trail
3,PARIS-13E-ARRONDISSEMENT,0.189655,1,48.828718,2.362468,Nouvelles Fleur de Lotus,48.828309,2.358412,Thai Restaurant
3,PARIS-13E-ARRONDISSEMENT,0.189655,1,48.828718,2.362468,Thai Royal,48.825140,2.362251,Thai Restaurant
3,PARIS-13E-ARRONDISSEMENT,0.189655,1,48.828718,2.362468,La Tropicale,48.831567,2.357190,Dessert Shop
3,PARIS-13E-ARRONDISSEMENT,0.189655,1,48.828718,2.362468,My Canh,48.824519,2.362318,Vietnamese Restaurant


In [84]:
#Cluster 2
asian_merged.loc[asian_merged['Cluster Labels'] == 2]

,Neighborhood,Asian Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,Le Verre Volé - Le Bistrot,48.872869,2.363669,Wine Bar
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,Hôtel du Nord,48.873443,2.363924,Restaurant
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,Restaurant Le Robinet d'Or,48.878025,2.364761,Bistro
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,Bopome,48.871701,2.362483,Asian Restaurant
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,La Cantine de Quentin,48.873945,2.363959,French Restaurant
...,...,...,...,...,...,...,...,...,...
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,Bombay Palace,48.878327,2.356484,Indian Restaurant
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,La Grange,48.875331,2.367360,Italian Restaurant
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,Midnight Hotel,48.877301,2.356188,Hotel
0,PARIS-10E-ARRONDISSEMENT,0.03,2,48.876029,2.361113,Sushi Rama,48.873197,2.358073,Japanese Restaurant
